# Introduction

L'objectif de ce projet était de réaliser une étude sur le marché des smartphones reconditionnés afin de déterminer quels smartphones ont un prix inférieur à celui du marché pour des caractéristiques données. 

Pour cela, j'ai scrapé le site de backmarket (https://www.backmarket.fr/), afin de récolter les prix et les caractéristiques et informations sur l'état des smartphones. J'ai lancé le scraping 2 fois étant donné que les annonces changent très régulièrement sur ce site.

Ensuite, j'ai combiné et nettoyé les 2 bases de données. Une analyse descriptive de la base nettoyée a également été faite, afin de mieux comprendre les différentes variables.

Enfin, les données ont été entrainé sur des modèles de machine learning afin de ressortir un modèle pour faire ressortir les smartphones ayant un prix en-dessous du prix prédit pour leurs caractéristiques.

# Scraping

## Fonction principale pour le scraping

In [1]:
from Scraping import main

In [ ]:
main(nom_fichier="data_complete_1.json")

In [ ]:
main(nom_fichier="data_complete_2.json")

La fonction `main` lance une session, en utilisant la classe Session, puis accepte les cookies et fait la recherche des smartphones. Ensuite, à chaque annonce cliquée, la classe Description permet de sauvegarder les caractéristiques importantes du smartphone et renvoit le tout dans un fichier `json`.

Il y a un seul argument à lui passer pour un scraping complet : celui du nom de sauvegarde de la base de données au format `.json`.

## Naviguer sur backmarket

Pour lancer la session :

In [2]:
from Scraping import Session

In [3]:
lien = "https://www.backmarket.fr/"
nav = Session(lien)

Pour agrandir la fenêtre :

In [4]:
nav.navigateur.maximize_window()

Pour accepter les cookies :

In [5]:
nav.accepter_cookies()

Pour chercher un bien matériel sur le site :

In [6]:
element = "smartphone"
nav.chercher_element(element)

Pour cliquer sur toutes les annonces d'une même page et récupérer le descriptif :

In [ ]:
nav.clique_annonce()

Pour parcourir les différentes pages de la recherche :

In [7]:
nav.change_page()

Pour récupérer le lien de l'annonce :

In [ ]:
lien = nav.navigateur.current_url

Pour quitter le navigateur proprement :

In [8]:
nav.navigateur.quit()

# Nettoyage données

In [1]:
from Nettoyage import recupere_bases_de_donnees_complete, nettoie_donnees, sauvegarde_csv
import os

In [2]:
path = os.path.abspath(os.getcwd())

Pour récupérer la base de donnée complète :

In [3]:
data = recupere_bases_de_donnees_complete(path + "\Donnees")

Ensuite, on nettoie les données :

In [4]:
data_final = nettoie_donnees(data)

In [5]:
data_final

,etat,marque,prix_euro,couleur,taille_ecran_pouce,capacite_stockage_Go,megapixel,systeme_exploitation,resolution_ecran,reseau,date_de_sortie,memoire,connecteur,double_sim,port_carte_SD,pliable,poids_g
0,État correct,apple,201.0,gris sidéral,4.7,64.0,12,iOS,750x1334,4G,2017,2.0,Lightning,Non,Non,Non,148
1,Très bon état,apple,213.0,gris sidéral,4.7,64.0,12,iOS,750x1334,4G,2017,2.0,Lightning,Non,Non,Non,148
2,Parfait état,apple,263.0,gris sidéral,4.7,64.0,12,iOS,750x1334,4G,2017,2.0,Lightning,Non,Non,Non,148
3,État correct,apple,358.0,noir,6.1,64.0,12,iOS,828x1792,4G,2018,3.0,Lightning,Non,Non,Non,194
4,Très bon état,apple,379.0,noir,6.1,64.0,12,iOS,828x1792,4G,2018,3.0,Lightning,Non,Non,Non,194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,Très bon état,samsung,699.0,noir,6.8,512.0,12,Android,1440x3040,4G,2019,12.0,USB-C,Non,Oui,Non,196
2500,Parfait état,samsung,999.0,noir,6.8,512.0,12,Android,1440x3040,4G,2019,12.0,USB-C,Non,Oui,Non,196
2501,Parfait état,samsung,969.0,lavande,6.7,128.0,12,Android,1080x2640,5G,2021,8.0,USB-C + Jack 3.5mm,Oui,Non,Oui,183
2502,État correct,orange,39.0,noir,5.0,8.0,5,Android,480x854,3G,2017,8.0,micro USB + Jack 3.5mm,Non,Oui,Non,169


Nous pouvons également sauvegarder en csv la base de données nettoyée :

In [6]:
sauvegarde_csv(path+"\Donnees", data_final)

'Base de données sauvegardée.'

La base de données finale comporte 2504 observations et 17 variables :
- etat
- marque
- prix_euro
- couleur
- taille_ecran_pouce
- capacite_stockage_Go
- megapixel
- systeme_exploitation
- resolution_ecran
- reseau
- date_de_sortie
- memoire
- connecteur
- double_sim
- port_carte_SD
- pliable
- poids_g

Pour plus d'informations sur les variables, il est conseillé d'aller regarder le fichier `analyse_descriptive`. De plus, le module `Tests` permet d'avoir une première vérification sur les fonctions de nettoyage pour chaque variable.

# Machine learning

Pour la partie de machine learning, la variable à expliquer est les prix en euro et les variables explicatives sont celles listées ci-dessus. Pour ce qui est de la variable `prix_euro`, j'ai décidé d'utiliser les déciles. Ce choix se porte sur le fait que dans l'analyse descriptive, on a pu remarquer que les prix semblaient plutôt suivre une loi normale centrée en 200. Prendre les déciles permet d'avoir assez de classes pour la prédiction, sans en avoir de trop. Cela permet notamment de capter les extrémités et de délimiter un peu plus les prix où la densité est la plus élevé, c'est-à-dire au niveau de 200.

## Apprentissage

In [6]:
from Machine_learning import (
    genere_configuration, chargement_configuration, affichage_configuration, 
    apprentissage_modele, modifie_dataframe, visualisation_apprentissage
)

In [7]:
import pandas as pd
import os
import pickle

In [8]:
path = os.path.abspath(os.getcwd())

Permet de générer la configuration dans un fichier yaml :

In [9]:
genere_configuration("config.yaml")

In [10]:
chargement_configuration("config.yaml")

Config(arbre_decision=Arbre(config_vide=False, longueur_arbre=[1, 3, 5, 7], nombre_noeuds=[2, 4, 6], parametre_complexite=[0.0, 1.0, 2.0]), bayesien=Bayesien(config_vide=False, alpha=[0.001, 0.01, 0.1, 1.0]), benet=Benet(config_vide=False, strategies=[<Strategie.PRIOR: 'prior'>, <Strategie.STRATIFIED: 'stratified'>, <Strategie.UNIFORM: 'uniform'>]), foret=Foret(config_vide=False, longueur_arbre=[1, 5, 10, 25], nombre_estimateurs=[10, 100, 250, 400]), k_voisins=KVoisins(config_vide=False, nombre_voisins=[1, 5, 11, 20], type_distance=[1, 2]), neurones=Neurones(config_vide=False, architecture=[[100], [50, 50]], alpha=[0.001, 0.1], max_iter=[500, 1000, 10000]), support_vecteurs=SupportVecteurs(config_vide=False, regularisation=[0.1, 1.0, 5.0]))

On visualise la configuration :

In [11]:
affichage_configuration("config.yaml")

Config(
    arbre_decision=Arbre(
        config_vide=False,
        longueur_arbre=[1, 3, 5, 7],
        nombre_noeuds=[2, 4, 6],
        parametre_complexite=[0.0, 1.0, 2.0]
    ),
    bayesien=Bayesien(config_vide=False, alpha=[0.001, 0.01, 0.1, 1.0]),
    benet=Benet(
        config_vide=False,
        strategies=[
            <Strategie.PRIOR: 'prior'>,
            <Strategie.STRATIFIED: 'stratified'>,
            <Strategie.UNIFORM: 'uniform'>
        ]
    ),
    foret=Foret(
        config_vide=False,
        longueur_arbre=[1, 5, 10, 25],
        nombre_estimateurs=[10, 100, 250, 400]
    ),
    k_voisins=KVoisins(
        config_vide=False,
        nombre_voisins=[1, 5, 11, 20],
        type_distance=[1, 2]
    ),
    neurones=Neurones(
        config_vide=False,
        architecture=[[100], [50, 50]],
        alpha=[0.001, 0.1],
        max_iter=[500, 1000, 10000]
    ),
    support_vecteurs=SupportVecteurs(config_vide=False, regularisation=[0.1, 1.0, 5.0])
)

On importe la base de données si ce n'est pas déjà fait et on modifie nos données pour qu'elles soient utilisables par la fonction `apprentissage` :

In [12]:
data = pd.read_csv(path + "\Donnees\complete_database.csv", index_col=0)

In [13]:
data_dummy = modifie_dataframe(data)

On lance les différentes validations croisées sur nos données. Nos modèles seront sauvegardés dans le dossier `Modele`.

In [17]:
resultats_apprentissage = apprentissage_modele("config.yaml", data_dummy)

Arbre de décision fini.

Bayesien fini.

DummyClassifier fini.

Forêt aléatoire finie.

k-plus proches voisins fini.

Réseaux de neurones fini.

Support vecteur fini.

Pour visualiser les résultats :

In [15]:
visualisation_apprentissage(resultats_apprentissage)

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Modele                 ┃ Score    ┃ Temps          ┃ Meilleur choix de Paramètres         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ DummyClassifier        │ 0.1065   │ 0.0016 minutes │                                      │
│ SVC                    │ 0.16881  │ 0.2581 minutes │ C=5.0                                │
│ MultinomialNB          │ 0.3637   │ 0.0045 minutes │ alpha=0.001                          │
│ MLPClassifier          │ 0.52288  │ 3.9902 minutes │ alpha=0.001,                         │
│                        │          │                │ hidden_layer_sizes=[100],            │
│                        │          │                │ max_iter=10000                       │
│ DecisionTreeClassifier │ 0.61608  │ 0.0543 minutes │ max_depth=5, max_leaf_nodes=6        │
│ KNeighborsClassifier   │ 0.77636  │ 0.098 minutes  │ n_neighbors=1                        │
│ RandomForestClassifier │ 0.85676  │ 1.1033 minutes │ max_depth=25, n_estimators=400       │
└────────────────────────┴──────────┴────────────────┴──────────────────────────────────────┘

Ici, il semblerait que notre meilleur modèle soit la forêt aléatoire. Ce modèle sera utilisé sur les données test afin de définir lequel est le meilleur.

### Sauvegarder et lire les résultats de l'apprentissage

Sauvegarder les résultats :

In [24]:
with open(path + "\Resultats_apprentissage.pkl", "wb") as fichier_modele:
    pickle.dump(resultats_apprentissage, fichier_modele)

Lire les résultats :

In [14]:
with open(path + "\Resultats_apprentissage.pkl", "rb") as fichier_modele:
    resultats_apprentissage = pickle.load(fichier_modele)

## Test

On importe les fonctions essentielles afin de comparer sur les données test nos modèles :

In [16]:
from Machine_learning import (
    teste_precision_modele, matrice_confusion_donnees_test
)

On récupère le meilleur modèle sauvegardé :

In [17]:
with open(path + "\Modele\Foret_aleatoire.pkl", "rb") as fichier_modele:
    foret_aleatoire = pickle.load(fichier_modele)

On teste la précision de nos modèles sur les données tests :

In [18]:
prediction = teste_precision_modele(data_dummy, foret_aleatoire)

In [19]:
prediction

{RandomForestClassifier(max_depth=25, n_estimators=400): 0.854632587859425}

Il n'y a pas de surapprentissage sur la forêt aléatoire. J'ai donc décidé de garder ce modèle. La matrice de confusion est la suivante :

In [20]:
print(matrice_confusion_donnees_test(data_dummy, foret_aleatoire))

[[52  5  0  0  0  0  0  0  0  0]
 [ 4 42  2  0  0  2  0  0  0  0]
 [ 2  6 54  4  0  0  0  0  0  0]
 [ 1  0 13 44  7  0  0  0  0  0]
 [ 0  0  4  6 53  0  5  0  0  0]
 [ 0  2  2  0  0 62  0  0  0  1]
 [ 0  0  0  1  7  0 65  6  0  0]
 [ 0  0  0  1  1  0  3 45  0  0]
 [ 0  1  0  0  0  0  0  3 61  0]
 [ 0  0  0  0  0  0  0  0  2 57]]


Nous pouvons remarquer que la majorité des valeurs semblent bien prédites et ce, pour toutes les classes.

## Modèle final

Une fois le meilleur modèle trouvé, nous l'appliquons aux données d'entrainement afin de pouvoir l'utiliser sur d'autres jeux de données.

In [21]:
from Machine_learning import renvoie_echantillons_test_train

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
modele = RandomForestClassifier(n_estimators=400, max_depth=25)

In [27]:
X_tr, _, y_tr, _ = renvoie_echantillons_test_train(data_dummy)

In [25]:
modele_final = modele.fit(X_tr, y_tr)

In [26]:
with open(path + "\Modele\Modele_final.pkl", "wb") as fichier_modele:
    pickle.dump(modele_final, fichier_modele)

# Et ensuite ?

Maintenant que nous avons déterminer notre meilleur modèle et qu'il a été appliqué à toutes les données, il faudrait refaire une phase de scraping afin de faire ressortir les annonces ayant un prix prédit strictement inférieur à la réalité.

Pour cela, il faudrait prédire les prix en utilisant les nouvelles données scrapées et comparer les valeurs prédites aux valeurs observées. Si le prix observé est strictement inférieur à l'intervalle prédit, dans ce cas, cette annonce a de fortes chances d'être une bonne affaire pour le consommateur.